In [346]:
import urllib
import numpy as np
import pandas as pd
import time

from bs4 import BeautifulSoup

# Список глав + распределение по частям

In [304]:
def headers(base_url, next_page):
    header_list = []
    
    while next_page:
        current_url = base_url + next_page
        page = urllib.request.urlopen(current_url)
        soup = BeautifulSoup(page)

        next_page = soup.select('.part_pages_sel + td')
        if next_page:
            next_page = next_page[0].find('a').get('href')
        else:
            try:
                next_page = soup.findAll('label', {'class': 'nav_label2'})[1].find('a').get('href')
            except IndexError:
                next_page = None

        content_text = soup.select('#content_text')
        if content_text:
            header_list.append(soup.select('.cttl')[0].text)
#             print(soup.select('.cttl')[0].text)
        
    return header_list

In [305]:
base_url = 'http://www.sql-tutorial.ru'
next_page = '/ru/book_typical_mistakes_select.html'

header_list = headers(base_url, next_page)

None


In [316]:
with open(f'./SQL_tutorial_parts.html', 'w') as output_file:
    for i in header_list:
        output_file.write(i.replace('\n', '') + '\n')

In [329]:
header_df = pd.DataFrame(header_list, columns=['chapter'])
header_df = header_df.assign(part=np.array([2] * 120 + [3] * 72 + [4] * 59 + [5] * 36 + [6] * 49))

# Сбор глав по частям

In [347]:
def pars_html(base_url, next_page):
    common_list = []
    
    with open(f'./SQL_tutorial_common.html', 'r') as input_file:
        for line in input_file:
            common_list.append(line)

    while next_page:
#     for i in range(1):
        current_url = base_url + next_page
        page = urllib.request.urlopen(current_url)
        soup = BeautifulSoup(page)

        next_page = soup.select('.part_pages_sel + td')
        if next_page:
            next_page = next_page[0].find('a').get('href')
        else:
            try:
                next_page = soup.findAll('label', {'class': 'nav_label2'})[1].find('a').get('href')
            except IndexError:
                next_page = None

        content_text = soup.select('#content_text')
        if content_text:
            header = soup.select('.cttl')[0].text
            npage = header_df[header_df['chapter'] == header]['part'].iloc[0]
            with open(f'./SQL_tutorial_part{npage}.html', 'a') as output_file:
                output_file.write(str(common_list[0]))
                output_file.write(str(soup.select('#parent_list')[0]) + '\n')
                output_file.write(str(soup.select('.cttl')[0]) + '\n')
                output_file.write(str(common_list[1]))
                output_file.write(str(content_text[0])[:-6] + '\n')

                add_content = soup.html
                if add_content.findNextSibling():
                    while add_content.findNextSibling().name != 'noindex':
                        if add_content.findNextSibling().name != 'div':
                            output_file.write(str(add_content.findNextSibling()) + '\n')
                            add_content = add_content.findNextSibling()
                            if add_content.findNextSibling() is None:
                                print(soup)
                                print('*'*80)
                        else:
                            break
                output_file.write('</div>')
                output_file.write(str(common_list[2]))
                
        time.sleep(10)

In [348]:
base_url = 'http://www.sql-tutorial.ru'
next_page = '/ru/book_typical_mistakes_select.html'

pars_html(base_url, next_page)